In [1]:
import numpy as np
import json
from roifile import ImagejRoi
import os

In [3]:


# Folder containing the images and zip files
data_folder = "df_images_roi"

# Base metadata
info = {
    "year": "2024",
    "version": "1",
    "description": "Exported from ROI files",
    "contributor": "",
    "url": "",
    "date_created": "2024-06-05T10:15:36+00:00"
}

licenses = [
    {
        "id": 1,
        "url": "https://creativecommons.org/publicdomain/zero/1.0/",
        "name": "Public Domain"
    }
]

categories = [
    {
        "id": 0,
        "name": "cross_sections",
        "supercategory": "none"
    }
]

# Prepare results
images = []
annotations = []
annotation_id = 0
image_id = 0

# Loop through all files
for filename in sorted(os.listdir(data_folder)):
    if filename.endswith(".jpg") or filename.endswith(".JPG"):
        image_path = os.path.join(data_folder, filename)
        base_name = os.path.splitext(filename)[0]
        zip_filename = f"{base_name}.zip"
        zip_path = os.path.join(data_folder, zip_filename)

        # Skip if no corresponding zip
        if not os.path.exists(zip_path):
            print(f"Warning: ZIP file not found for {filename}")
            continue

        # Load ROIs
        rois = ImagejRoi.fromfile(zip_path)

        # Append image metadata
        image_metadata = {
            "id": image_id,
            "license": 1,
            "file_name": filename,
            "height": 2048,
            "width": 2048,
            "date_captured": "2024-06-05T10:15:36+00:00"
        }
        images.append(image_metadata)

        # Append annotations
        for roi in rois:
            coords = roi.coordinates().tolist()
            x_min = min(pt[0] for pt in coords)
            y_min = min(pt[1] for pt in coords)
            width_bbox = max(pt[0] for pt in coords) - x_min
            height_bbox = max(pt[1] for pt in coords) - y_min
            area = width_bbox * height_bbox

            annotations.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": 0,
                "bbox": [x_min, y_min, width_bbox, height_bbox],
                "area": area,
                "segmentation": [coord for point in coords for coord in point],
                "iscrowd": 0
            })
            annotation_id += 1

        image_id += 1

# Combine everything into one dict
coco_annotation = {
    "info": info,
    "licenses": licenses,
    "categories": categories,
    "images": images,
    "annotations": annotations
}

# Save to file
output_json_path = os.path.join(data_folder, "annotations.json")
with open(output_json_path, "w") as f:
    json.dump(coco_annotation, f, indent=2)

print(f"Saved annotations to {output_json_path}")


Saved annotations to df_images_roi\annotations.json
